## Project Advanced Modelisation : Test Unequilibrated data -> Split the Data

In [87]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from statsmodels.imputation import mice
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, confusion_matrix
import operator
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data = pd.read_csv("GV_MKT_data_scoring_appetence.csv",sep=",")

**Valeurs aberrantes négatives**

In [7]:
ind = np.array(np.where(data < 0))[0]
data.drop(ind, axis=0, inplace=True)
data = data.reset_index()

In [6]:
data

,index,IDF,OUV,MTTDRNCRE,MTTDRNAGI,MOIDRNCRE,DELMOYCRE,MTTRSS,NBRPRSCHA,DURCRE,NBRCRE
0,0,22,0,1263,253,72,216.00,2202.0,2,6,1
1,1,46,0,500,100,1,6.00,1605.0,2,62,1
2,2,52,0,500,100,1,6.00,865.0,2,6,4
3,3,92,0,1663,83,1,6.00,4294.0,1,59,1
4,4,132,0,2068,290,49,118.58,1092.0,2,9,1
...,...,...,...,...,...,...,...,...,...,...,...
5339,5493,99866,0,11111,2222,1,6.00,4441.0,1,37,12
5340,5494,99881,0,9446,1889,54,82.08,3510.0,3,72,1
5341,5495,99881,0,500,100,40,120.00,1426.0,2,21,9
5342,5496,99964,0,918,204,49,147.00,300.0,1,4,8


In [293]:
ind1 = data[data["OUV"] == 1].index
ind1 = ind1[0:375]
ind1.shape

(375,)

In [287]:
ind0 = data[data["OUV"] == 0].index
ind0 = ind0[0:450]
ind0.shape

(450,)

In [294]:
df0 = data.iloc[ind0,:]

In [295]:
df1 = data.iloc[ind1,:]

In [296]:
liste_concat = [df0, df1]
df_eq =pd.concat(liste_concat)
print(df_eq.shape)
df_eq.reset_index
df_eq = df_eq.iloc[:,3:12]
df_eq = df_eq.sample(frac=1).reset_index(drop=True)
df_eq.head()

(825, 12)


,OUV,MTTDRNCRE,MTTDRNAGI,MOIDRNCRE,DELMOYCRE,MTTRSS,NBRPRSCHA,DURCRE,NBRCRE
0,0,6197,310,20,60.00,4319.0,2,104,1
1,1,500,100,48,33.60,5935.0,3,40,12
2,0,5593,447,24,85.20,2682.0,1,6,1
3,0,6403,640,24,151.92,3966.0,2,24,1
4,0,500,70,20,167.80,2474.0,1,28,6


In [67]:
logit_mod = smf.logit("OUV ~ MTTDRNCRE + MTTDRNAGI + MOIDRNCRE + DELMOYCRE + MTTRSS + NBRPRSCHA + DURCRE + NBRCRE", data=df_eq)
logit_res = logit_mod.fit()

logit_res.summary()

print(logit_res.prsquared)
print(logit_res.llr_pvalue)
print(np.exp(logit_res.params))
print(logit_res.fittedvalues[:10])
p = logit_res.predict()
print(p)


#y_pred = (p > 0.10).astype(int)
#print(y_pred)

for i in np.arange(0.01, 1, 0.01):
    y_pred = (p > i).astype(int)
    print(i)
    print(logit_res.pred_table(i))
    #print(f1_score(df_eq["OUV"].values, y_pred))

Optimization terminated successfully.
         Current function value: 0.574560
         Iterations 6
0.17108557950721004
9.273726507043297e-42
Intercept    2.189426
MTTDRNCRE    0.999812
MTTDRNAGI    1.001464
MOIDRNCRE    1.024846
DELMOYCRE    0.985749
MTTRSS       1.000092
NBRPRSCHA    0.673660
DURCRE       0.983788
NBRCRE       1.040112
dtype: float64
0   -1.063393
1   -0.842058
2    0.123086
3   -0.394115
4   -0.478244
5    0.111976
6   -1.348746
7   -2.417029
8   -1.732639
9   -2.344464
dtype: float64
[0.25666167 0.30110152 0.53073283 0.40272708 0.38266676 0.52796482
 0.2060755  0.08188332 0.15025038 0.08750681 0.31173418 0.34645278
 0.08509023 0.22765715 0.47262279 0.6178505  0.58717312 0.63185815
 0.29122977 0.24889509 0.45288063 0.4482734  0.97912245 0.24833301
 0.2270268  0.47861455 0.36936644 0.48912226 0.64410542 0.39644537
 0.53176462 0.87139392 0.23486659 0.4874106  0.46659511 0.13828828
 0.4146212  0.55560319 0.41367485 0.43036133 0.1761374  0.53877786
 0.88047354 0.07279

In [65]:
print(logit_res.pred_table(0.43))

[[258. 192.]
 [ 73. 377.]]


**DF**

In [72]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

rf = DecisionTreeClassifier(random_state=42, criterion = "gini")

X = (data.iloc[:, 4:11]).astype("float64")
y = data.iloc[:, 3]
print(X.dtypes)

# Fit the GridSearchCV object to the data
rf.fit(X, y)

# Print the best hyperparameters

scores = cross_val_score(rf, X, y, scoring = "f1")
scores.mean()

#y_pred = xgbc.predict(X_test)

#test_score = f1_score(y_test, y_pred)
#print('Test score:', test_score)

MTTDRNCRE    float64
MTTDRNAGI    float64
MOIDRNCRE    float64
DELMOYCRE    float64
MTTRSS       float64
NBRPRSCHA    float64
DURCRE       float64
dtype: object


0.17684804484570835

In [284]:
from sklearn.metrics import accuracy_score
rf = DecisionTreeClassifier(random_state=42, criterion = "gini")

X = (df_eq.iloc[:, 1:9]).astype("float64")
y = df_eq.iloc[:, 0]
print(X.dtypes)

X1 = (data.iloc[:, 4:11]).astype("float64")
y1 = data.iloc[:, 3]

# Fit the GridSearchCV object to the data
rf.fit(X, y)

# Print the best hyperparameters

scores = cross_val_score(rf, X1, y1, scoring = "accuracy")
scores.mean()

x_samp = data.sample(n=900)
x_samp = x_samp.iloc[:,3:12]
y_samp = x_samp.iloc[:,0]
y_samp = pd.DataFrame(y_samp)
x_samp = x_samp.iloc[:,1:12]


y_pred = rf.predict(x_samp)
print(confusion_matrix(y_samp, y_pred))
print(accuracy_score(y_samp, y_pred))




MTTDRNCRE    float64
MTTDRNAGI    float64
MOIDRNCRE    float64
DELMOYCRE    float64
MTTRSS       float64
NBRPRSCHA    float64
DURCRE       float64
NBRCRE       float64
dtype: object
[[664 171]
 [ 28  37]]
0.7788888888888889


In [298]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42, criterion = "gini")

X = (df_eq.iloc[:, 1:9]).astype("float64")
y = df_eq.iloc[:, 0]
print(X.dtypes)

X1 = (data.iloc[:, 4:11]).astype("float64")
y1 = data.iloc[:, 3]

# Fit the GridSearchCV object to the data
rf.fit(X, y)

# Print the best hyperparameters

scores = cross_val_score(rf, X1, y1, scoring = "accuracy")
scores.mean()

x_samp = data.sample(n=df_eq.shape[0])
x_samp = x_samp.iloc[:,3:12]
y_samp = x_samp.iloc[:,0]
y_samp = pd.DataFrame(y_samp)
x_samp = x_samp.iloc[:,1:12]


y_pred = rf.predict(x_samp)
print(confusion_matrix(y_samp, y_pred))
print(accuracy_score(y_samp, y_pred))

MTTDRNCRE    float64
MTTDRNAGI    float64
MOIDRNCRE    float64
DELMOYCRE    float64
MTTRSS       float64
NBRPRSCHA    float64
DURCRE       float64
NBRCRE       float64
dtype: object
[[597 163]
 [  4  61]]
0.7975757575757576


In [270]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter space
param_dist = {'n_estimators': randint(1, 50),
              'max_depth': randint(1, 5),
              'min_samples_split': randint(1, 10),
              'min_samples_leaf': randint(1, 10),
              'bootstrap': [True, False]}

# Create a Random Forest Classifier
rf = RandomForestClassifier()
X = (df_eq.iloc[:, 1:9]).astype("float64")
y = df_eq.iloc[:, 0]
print(X.dtypes)
X1 = (data.iloc[:, 4:11]).astype("float64")
y1 = data.iloc[:, 3]

# Create a RandomizedSearchCV object
random_search = RandomizedSearchCV(rf, param_distributions=param_dist, scoring = "accuracy", n_iter=40, cv=6, random_state=42)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X, y)



# Print the best hyperparameters

scores = cross_val_score(random_search, X1, y1, scoring = "accuracy")
scores.mean()

x_samp = data.sample(n=df_eq.shape[0])
x_samp = x_samp.iloc[:,3:12]
y_samp = x_samp.iloc[:,0]
y_samp = pd.DataFrame(y_samp)
x_samp = x_samp.iloc[:,1:12]


y_pred = random_search.predict(x_samp)
print(confusion_matrix(y_samp, y_pred))
print(accuracy_score(y_samp, y_pred))


MTTDRNCRE    float64
MTTDRNAGI    float64
MOIDRNCRE    float64
DELMOYCRE    float64
MTTRSS       float64
NBRPRSCHA    float64
DURCRE       float64
NBRCRE       float64
dtype: object
[[421 165]
 [ 24  20]]
0.7
